# scSpatial Seurat

In [ ]:
# install.packages("Seurat")

In [ ]:
# Library import

library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)
library(magrittr)

set.seed(1)

# Multicore CPU usage activation - Future
library(future)

plan("multicore", workers = 10) 
# Set RAM Size to 3/4 of total RAM
options(future.globals.maxSize = 16000 * 1024^2)

future.seed=TRUE # Removes future-generated statistical errors

In [ ]:
library(data.table)
# Load data
cell_by_gene <- fread("cell_by_gene.csv", data.table = FALSE)
cell_metadata <- fread("cell_metadata.csv", data.table = FALSE)

# Extract cell barcodes and gene names
cell_barcodes <- cell_by_gene[, 1]  # First column contains cell barcodes
gene_names <- colnames(cell_by_gene)[-1]  # Gene names are column names

# Ensure unique gene names
gene_names <- make.unique(gene_names)  

# Transpose matrix
cell_by_gene <- t(cell_by_gene[, -1])  # Remove first column (cell barcodes) before transposing

# Assign correct row & col names
rownames(cell_by_gene) <- gene_names  # Genes as row names
colnames(cell_by_gene) <- make.unique(cell_barcodes)  # Ensure unique cell names

# Convert to matrix
cell_by_gene <- as.matrix(cell_by_gene)

# Create Seurat object
Seurat <- CreateSeuratObject(counts = cell_by_gene)

# Fix metadata cell names
cell_metadata$V1 <- gsub("_\\d+$", "", cell_metadata$V1)  # Remove suffixes like "_10" if needed
rownames(cell_metadata) <- make.unique(cell_metadata$V1)  # Ensure unique rownames

# Remove first column from metadata
cell_metadata <- cell_metadata[, -1]

# Add metadata
Seurat <- AddMetaData(Seurat, metadata = cell_metadata)

Seurat

In [ ]:
head(Seurat@meta.data)

In [ ]:
VlnPlot(Seurat, features = "nFeature_RNA")
VlnPlot(Seurat, features = "nCount_RNA")

In [ ]:
Seurat <- NormalizeData(Seurat, normalization.method = "LogNormalize", scale.factor = 10000)
all.genes <- rownames(Seurat)
Seurat <- ScaleData(Seurat, features = all.genes)

Seurat

In [ ]:
distance_clusters <- read.csv("cluster_info.csv")
Seurat <- AddMetaData(Seurat, metadata = distance_clusters$Cluster, col.name = "CellType")
Seurat <- SetIdent(Seurat, value = "CellType")

Seurat

In [ ]:
Seurat@meta.data$center_x <- as.numeric(Seurat@meta.data$center_x)
Seurat@meta.data$center_y <- as.numeric(Seurat@meta.data$center_y)

# Plot clusters with black background
ggplot(Seurat@meta.data, aes(x = center_x, y = center_y, color = as.factor(CellType))) +
  geom_point(size = 1) +
  scale_color_manual(values = rainbow(length(unique(Seurat$CellType)))) + # Custom colors
  theme_void() +  # Remove axis
  theme(panel.background = element_rect(fill = "black", color = "black"), 
        plot.background = element_rect(fill = "black"),
        legend.text = element_text(color = "white"),
        legend.title = element_text(color = "white")) +
  labs(title = "Spatial Plot of Clusters", color = "CellType")

ggsave(
  "Spatial Cluster Plot.pdf",
  plot = last_plot(),
  device = "pdf",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 42,
  height = 40,
  units = "cm",
  dpi = 10000,
  limitsize = TRUE,
#   bg = NULL,
)

In [ ]:
library(ggplot2)

# Convert spatial coordinates to numeric
Seurat@meta.data$center_x <- as.numeric(Seurat@meta.data$center_x)
Seurat@meta.data$center_y <- as.numeric(Seurat@meta.data$center_y)

# Filter for clusters 29, 3, and 6
subset_meta <- Seurat@meta.data[Seurat@meta.data$CellType %in% c(29, 20, 15), ]

# Define custom colors for selected clusters
custom_colors <- c("29" = "purple", "20" = "green", "15" = "yellow")  # Customize as needed

# Plot clusters with black background
ggplot(subset_meta, aes(x = center_x, y = center_y, color = as.factor(CellType))) +
  geom_point(size = 1) +
  scale_color_manual(values = custom_colors) + # Use custom colors
  theme_void() +  # Remove axis
  theme(panel.background = element_rect(fill = "black", color = "black"), 
        plot.background = element_rect(fill = "black"),
        legend.text = element_text(color = "white"),
        legend.title = element_text(color = "white")) +
  labs(title = "Spatial Plot of Selected Clusters", color = "Cluster")

ggsave(
  "Spatial SubCluster Plot.pdf",
  plot = last_plot(),
  device = "pdf",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 42,
  height = 40,
  units = "cm",
  dpi = 10000,
  limitsize = TRUE,
#   bg = NULL,
)

In [ ]:
# Define FOV boundaries
x_min <- 3000
x_max <- 8000
y_min <- 7500
y_max <- 12500

# Filter cells within the FOV
subset_cells <- rownames(Seurat@meta.data[
  Seurat@meta.data$center_x >= x_min & 
  Seurat@meta.data$center_x <= x_max & 
  Seurat@meta.data$center_y >= y_min & 
  Seurat@meta.data$center_y <= y_max, 
])

# Subset Seurat object
Seurat_FOV <- subset(Seurat, cells = subset_cells)

# Check dimensions of the new object
print(dim(Seurat_FOV))


In [ ]:
Seurat_FOV@meta.data$center_x <- as.numeric(Seurat_FOV@meta.data$center_x)
Seurat_FOV@meta.data$center_y <- as.numeric(Seurat_FOV@meta.data$center_y)

# Plot clusters with black background
ggplot(Seurat_FOV@meta.data, aes(x = center_x, y = center_y, color = as.factor(CellType))) +
  geom_point(size = 2) +
  scale_color_manual(values = rainbow(length(unique(Seurat$CellType)))) + # Custom colors
  theme_void() +  # Remove axis
  theme(panel.background = element_rect(fill = "black", color = "black"), 
        plot.background = element_rect(fill = "black"),
        legend.text = element_text(color = "white"),
        legend.title = element_text(color = "white")) +
  labs(title = "Spatial Plot of Clusters", color = "CellType")

ggsave(
  "Spatial FOV Cluster Plot.pdf",
  plot = last_plot(),
  device = "pdf",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 42,
  height = 40,
  units = "cm",
  dpi = 10000,
  limitsize = TRUE,
#   bg = NULL,
)

In [ ]:
levels(Seurat) <- c('1','2','3','4','5','6','7','8','9','10',
                    '11','12','13','14','15','16','17','18','19','20',
                   '21','22','23','24','25','26','27','28','29','30','31')

new.cluster.ids <- c('1','2','3','4','5','6','7','8','9','10',
                    '11','12','13','14','15','16','17','18','19','20',
                   '21','22','23','24','25','26','27','28','29','30','31')

names(new.cluster.ids) <- levels(Seurat)

Seurat <- RenameIdents(Seurat, new.cluster.ids)

In [ ]:
deg0 <- FindMarkers(Seurat, ident.1 = '29',ident.2 = '1', only.pos = FALSE, logfc.threshold = 1., min.pct = 0)
deg0 <- deg0[order(deg0$avg_log2FC,decreasing=TRUE),]
deg0

In [ ]:
cluster_features = c('ACOT7', 'RERE', 'DDI2',
                     'NADK', 'TP73', 'NPHP4',
                     'FAAP20', 'LRRC47', 'C1orf174', 
                     'MIB2', 'CDK11B', 'SKI',
                     'SLC35E2B', 'TNFRSF14', 'RSRP1',
                     'COL8A2', 'PODN', 'PTGER3',
                     'MXRA8', 'MEGF6', 'ID3',
                     'KIF1B', 'DHRS3', 'SYNC', 
                     'AQP1', 'RAMP3', 'PODXL', 
                     'TNFRSF1B', 'ALPL', 'ASAP3',
                     'C1QA', 'C1QC', 'C1QB', 
                     'CCN1', 'DPT', 'PDLIM3', 
                     'PLA2G2A', 'ACKR1','CD34',
                     'LCE3E', 'LCE3D', 'IVL',
                     'GRHL3', 'PDZK1IP1', 'CRCT1',
                     'HSPB7', 'S100A1', 'ATP1A2',
                     'SLC2A5', 'PADI2', 'TCEA3',
                     'DVL1', 'AJAP1', 'CASZ1',
                     'PLCH2', 'TPRG1L', 'GPR153',
                     'B3GALT6', 'AURKAIP1', 'KLHL21',
                     'ALDH4A1', 'CNKSR1', 'CDC20',
                     'NOC2L', 'HES4', 'UBE2J2', 
                     'PRKCZ', 'PER3', 'UBE4B',
                     'ACAP3', 'VWA1', 'WRAP73',
                     'COL3A1', 'FN1', 'COL6A3', 
                     'ENO1', 'MICOS10', 'NBL1',
                     'AGRN', 'CTNNBIP1', 'MFAP2',
                     'INTS11', 'CCNL2', 'SSU72',
                     'IGKC','IGLC1', 'IGKV4-1','S100A8','S100A9', 
                     'ICMT', 'ATP5IF1', 'MAP7D1',
                     'IFI44L', 'IFI44', 'GBP5'
                     )

library(viridis)
DotPlot(Seurat, features = cluster_features) + RotatedAxis() + scale_colour_viridis() +DarkTheme()

# Save the last plot as a pdf

ggsave(
  "Dot plot.pdf",
  plot = last_plot(),
  device = "pdf",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 60,
  height = 25,
  units = "cm",
  dpi = 10000,
  limitsize = TRUE,
#   bg = NULL,
)